# INNIO Fleet Analysis

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
import matplotlib.pyplot as plt
import bokeh
from collections import namedtuple
from pprint import (pprint as pp, pformat as pf)
import time
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
import ipywidgets as widgets
import arrow

import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML
from dfsm import msgFSM

### Filter Engines from installed fleet 

In [7]:
def sfun(x):
    return all([
            ("Forsa Hartmoor" in str(x['IB Site Name'])), 
            (x['OperationalCondition'] != 'Decommissioned')
        ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
pname = 'persist.json'
try:
    persist_dialog = dmyplant2.load_json(pname)
except FileNotFoundError:        
    persist_dialog = {
        'dl': 0,
        'modes': [
                {'name':'???','value':False },
                {'name':'OFF','value':False },
                {'name':'MANUAL','value':False},
                {'name':'AUTO','value':True }],
        'starts': [   
                {'name':'True','value':True },
                {'name':'False','value':False }]   
    }
    dmyplant2.save_json(pname,persist_dialog)    
fleet.T;

In [18]:
# Dialog
ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
ddl = [(m, i) for i, m in enumerate(ddl)]
#dl=widgets.Dropdown(options = ddl,value=0,description='Engine:',layout={'width':'max-content'},disabled=False)
dl=widgets.Dropdown(options = ddl,value=persist_dialog['dl'],description='Engine:',layout={'width':'auto'},disabled=False)
out = widgets.Output(layout={'height':'auto', 'width':'auto'})
rmodes = persist_dialog['modes']
service_selector_modes = [] 
modes = []
for mm in rmodes:
    service_selector_modes.append(widgets.Checkbox(
        value=mm['value'], 
        description=mm['name'],
        layout={'width':'auto'},
        disabled=False,indent=False))

rsucc = persist_dialog['starts']
successful_starts = []; 
for rs in rsucc:
    successful_starts.append(widgets.Checkbox(
            value=rs['value'],
            description=rs['name'],
            layout={'width':'auto'},
            disabled=False,indent=False))

app = AppLayout(  header=dl,
            #left_sidebar=Button(description='center', button_style='warning', layout=Layout(height='auto', width='auto')),
            left_sidebar=None,
            center=HBox([
                VBox(service_selector_modes),
                VBox(successful_starts),
                out
            ]),
            right_sidebar=None,
            footer = None,
            #footer = Button(description='footer', button_style='success', layout=Layout(height='auto', width='auto')),
            pane_widths=['10px',1,'10px'],
            pane_heights=['40px','250px','40px']
            )
display(app)

AppLayout(children=(Dropdown(description='Engine:', index=1, layout=Layout(grid_area='header', width='auto'), …

In [21]:
# Load Engine Data and execute FSM Run 1 - display Results 
persist_dialog = {'dl':dl.value, 'modes': [{'name':cb.description, 'value':cb.value} for cb in service_selector_modes],
                'starts': [{'name':cb.description, 'value':cb.value} for cb in successful_starts]}
dmyplant2.save_json(pname,persist_dialog)

e=dmyplant2.Engine.from_fleet(mp,motor:=fleet.iloc[dl.value])
fsm = msgFSM(e, p_from=e['Commissioning Date'], p_to=arrow.now(), frompickle=True)
fsm.run1(enforce=False) # run Finite State Machine
fsm.store()
out.clear_output()
with out:
    fsm.summary_out()
rdf = fsm.result
modes = [rmodes[i]['name'] for i,v in enumerate(service_selector_modes) if v.value]
success = [rsucc[i]['value'] for i,v in enumerate(successful_starts) if v.value]
rda = rdf[:].reset_index(drop='index')
rda['count_alarms'] = rda.apply(lambda x: len(x['alarms']), axis=1)
rda['count_warnings'] = rda.apply(lambda x: len(x['warnings']), axis=1)
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))]
display(HTML(pd.DataFrame.from_dict(e.dash, orient='index').T.to_html(escape=False, index=False)))
print(modes, success)
rda[fsm.filters['filter_content'] + fsm.filters['filter_alarms_and_warnings']].round(2)

HTML(value='<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Na…

['???', 'OFF', 'MANUAL', 'AUTO'] [True]


,success,mode,startpreparation,starter,hochlauf,idle,synchronize,loadramp,cumstarttime,targetoperation,count_alarms,count_warnings
4,True,AUTO,69.99,4.13,25.12,6.26,55.72,73.89,235.11,0 days 00:16:47,0,1
6,True,AUTO,60.53,3.84,19.07,5.55,45.95,65.78,200.73,0 days 00:00:52,0,0
7,True,AUTO,60.64,3.63,18.79,5.35,40.35,128.01,256.77,0 days 00:13:21,0,0
10,True,AUTO,60.63,3.83,19.17,5.75,60.33,127.37,277.09,0 days 00:03:22,0,0
15,True,AUTO,63.85,3.63,8.18,6.46,63.66,127.49,273.28,0 days 00:12:28,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
103,True,AUTO,101.27,4.04,23.71,4.84,48.66,98.35,280.86,0 days 05:13:31,0,0
104,True,AUTO,102.80,3.93,23.40,4.64,1.51,143.34,279.62,0 days 03:13:30,0,0
105,True,AUTO,100.13,3.74,23.63,4.44,46.84,98.79,277.57,0 days 01:43:32,0,0
106,True,AUTO,101.96,3.94,24.62,4.74,60.43,84.99,280.68,0 days 01:33:30,0,0


In [ ]:
# Execute FSM Run 2
rda = fsm.run2(rda)
fsm.store()

In [ ]:
rdb = rda
#rdb = rda[(rda['mode'].isin(['AUTO','MANUAL'])) & ((rda['count_alarms'] > 0) | (rda['count_warnings'] > 0))]
#rdb = rda[rda['synchronize'] < 15.0]
rdb[fsm.filters['run2filter_content'] + fsm.filters['filter_alarms_and_warnings']].round(2)

In [ ]:
display(_=rdb[['startpreparation','hochlauf','idle','synchronize','loadramp','maxload','ramprate','cumstarttime']].hist(bins=30,figsize=(20,20)))
display(rdb[['startpreparation','hochlauf','idle','synchronize','loadramp','maxload','ramprate','cumstarttime']].describe().round(2))

In [ ]:
startversuch = rdb.iloc[1];
von_dt=pd.to_datetime(startversuch['starttime']); von=int(von_dt.timestamp())
bis_dt=pd.to_datetime(startversuch['endtime']); bis=int(bis_dt.timestamp())

ftitle = f"{fsm._e} ----- Start {startversuch.name} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
print(ftitle, end=' ')
print(f"von: {von_dt.strftime('%d.%m.%Y %H:%M:%S')} = {von} bis: {bis_dt.strftime('%d.%m.%Y %H:%M:%S')} = {bis}")
vset = fsm._data_spec + ['Hyd_PressCrankCase','Hyd_PressOilDif','Hyd_PressOil','Hyd_TempOil']
data = fsm.get_cycle_data(startversuch, max_length=None, min_length=None, cycletime=1, silent=False, p_data=vset)
data

In [ ]:
dset = [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue'},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange'},
    {'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black'},
    {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown'},
    {'col':['Hyd_TempOil'],'ylim': [0, 110], 'color':'#2171b5'}
]
fsm.disp_result(startversuch)
al_lines = fsm.disp_alarms(startversuch)
w_lines = fsm.disp_warnings(startversuch)
fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(16,10), style='line', line_width=0)


In [ ]:
x = fig
y = 4
pass

In [ ]:
dmyplant2.add_dbokeh_vlines(al_lines,fig,line_color='purple', line_dash='dashed', line_alpha=1, line_width=2)
dmyplant2.add_dbokeh_vlines(w_lines,fig,line_color='brown', line_dash='dashed', line_alpha=1, line_width=2)
dmyplant2.add_dbokeh_vlines(fsm.states_lines(startversuch),fig,line_color='red', line_dash='solid', line_alpha=0.4)
bokeh.plotting.show(fig)

In [ ]:
# pure matplotlib chart
# fig, ax, axes = dmyplant2.chart(data, dset, figsize=(14,8), title=ftitle)
# dmyplant2.add_vlines(fsm.states_lines(startversuch), ax, color='gray', linestyle="-.")

In [ ]:
# Edge detection algorithm speed - visualization
# dset = [
#     {'col':['Various_Values_SpeedAct','Various_Values_SpeedAct_right'], 'ylim':(0,20000)},
#     {'col':['helpline_right'], 'color':'gray','ylim':(0,20000)}
# ]
# # pr, _ = fsm.detect_edge_right(data, 'Various_Values_SpeedAct', right=startversuch['endtime'])
# # fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)
# pr, ndata = fsm.detect_edge_right(data, 'Various_Values_SpeedAct', right=startversuch['endtime'])
# fig = dmyplant2.dbokeh_chart(ndata, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)
# dmyplant2.add_dbokeh_vlines([pr.loc], fig,line_color='red', line_dash='solid', line_alpha=0.4)
# bokeh.plotting.show(fig)

In [ ]:
# Edge detection algorithm speed - visualization
# dset = [
#     {'col':['Various_Values_SpeedAct','Various_Values_SpeedAct_left','Various_Values_SpeedAct_right','helpline_left','helpline_right'], '_ylim':(0,4000)},
# ]
# pr, ndata = fsm.detect_edge_right(data, 'Various_Values_SpeedAct', right=startversuch['endtime'])
# pl, ndata = fsm.detect_edge_left(ndata, 'Various_Values_SpeedAct', left=startversuch['starttime'])
# fig = dmyplant2.dbokeh_chart(ndata, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)
# # pl, _ = fsm.detect_edge_left(data, 'Various_Values_SpeedAct', left=startversuch['starttime'])
# # fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)
# dmyplant2.add_dbokeh_vlines([pl.loc], fig,line_color='red', line_dash='solid', line_alpha=0.4)
# dmyplant2.add_dbokeh_vlines([pr.loc], fig,line_color='red', line_dash='solid', line_alpha=0.4)
# bokeh.plotting.show(fig)

In [ ]:
# Edge detection algorithm power - visualization
# dset = [
#     {'col':['Power_PowerAct','Power_PowerAct_left','Power_PowerAct_right'], '_ylim':(0,40000)}
# ]
# #pl, _ = fsm.detect_edge_left(data, 'Power_PowerAct', left=startversuch['starttime'])
# pl, _ = fsm.detect_edge_left(data, 'Power_PowerAct', startversuch=startversuch)
# pr, _ = fsm.detect_edge_right(data, 'Power_PowerAct', right=startversuch['endtime'])
# fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(6,3), style='line', line_width=0)
# dmyplant2.add_dbokeh_vlines([pl.loc], fig,line_color='red', line_dash='solid', line_alpha=0.4)
# dmyplant2.add_dbokeh_vlines([pr.loc], fig,line_color='red', line_dash='solid', line_alpha=0.4)
# bokeh.plotting.show(fig)

In [ ]:
# # Edge detection algorithm power & speed - visualization
# dset = [
#     {'col':['Power_PowerAct','Various_Values_SpeedAct'], 'ylim':(0,30000)},
#     {'col':['Power_PowerAct_left','Power_PowerAct_right','Various_Values_SpeedAct_left', 'Various_Values_SpeedAct_right'], 'ylim':(0,30000)}
#     ]
# pl, _ = fsm.detect_edge_left(data, 'Power_PowerAct', startversuch)
# pr, _ = fsm.detect_edge_right(data, 'Power_PowerAct', startversuch)
# sl, _ = fsm.detect_edge_left(data, 'Various_Values_SpeedAct', startversuch)
# sr, _ = fsm.detect_edge_right(data, 'Various_Values_SpeedAct', startversuch)
# #fig, ax, axes = dmyplant2.chart(data, dset, figsize=(14,8), title=ftitle)
# fsm.disp_result(startversuch)
# al_lines = fsm.disp_alarms(startversuch)
# w_lines = fsm.disp_warnings(startversuch)

# fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)
# dmyplant2.add_dbokeh_vlines(al_lines,fig,line_color='purple', line_dash='dashed', line_alpha=1, line_width=2)
# dmyplant2.add_dbokeh_vlines(w_lines,fig,line_color='brown', line_dash='dashed', line_alpha=1, line_width=2)
# dmyplant2.add_dbokeh_vlines(fsm.states_lines(startversuch), fig,line_color='red', line_dash='solid', line_alpha=0.4)
# lcol='blue'
# dmyplant2.add_dbokeh_vlines([sl.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
# dmyplant2.add_dbokeh_vlines([sr.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
# dmyplant2.add_dbokeh_vlines([pl.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
# dmyplant2.add_dbokeh_vlines([pr.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
# bokeh.plotting.show(fig)

In [ ]:
# iterate over Result
p_dat = fsm._data_spec + ['Hyd_PressCrankCase','Hyd_PressOilDif','Hyd_PressOil','Hyd_TempOil']
dset = [
    {'col':['Power_PowerAct'], '_ylim':(0,5000), 'color':'red'},
    {'col':['Various_Values_SpeedAct'],'_ylim': [0, 2500], 'color':'blue'},
    {'col':['Hyd_PressCrankCase'],'_ylim': [-100, 100], 'color':'orange'},
    {'col':['Hyd_PressOilDif'],'_ylim': [0, 2], 'color':'black'},
    {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown'},
    {'col':['Hyd_TempOil'],'_ylim': [60, 110], 'color':'#2171b5'}
]
def crawl_result(rda, srange = None, p_figsize=(16,10)):
        if srange == None:
                srange = [s['index'] for i,s in rda.iterrows()] # too much memory needed !!
                #srange = [s for s in range(2)] # limit to 2 in the interim 
        else:
                if isinstance(srange,int):
                        srange = [srange]
                elif not isinstance(srange,list):
                        raise ValueError(f"crawl_result: invalid parameter! srange={srange}")
        print(srange)
        for ii,startversuch in rda.iterrows():
                if startversuch['index'] in srange:
                        data = fsm.get_cycle_data(startversuch, max_length=None, min_length=None, cycletime=1, silent=True, p_data=p_dat)
                        print(data.shape)
                        pl, _ = fsm.detect_edge_left(data, 'Power_PowerAct', startversuch)
                        pr, _ = fsm.detect_edge_right(data, 'Power_PowerAct', startversuch)
                        sl, _ = fsm.detect_edge_left(data, 'Various_Values_SpeedAct', startversuch)
                        sr, _ = fsm.detect_edge_right(data, 'Various_Values_SpeedAct', startversuch)
                        # code ?
                        ml = (data.iloc[-1]['time'] - data.iloc[0]['time']) // 1000
                        sv_lines = [v for v in startversuch[fsm.filters['vertical_lines_times']]]
                        nsv_lines = [v for v in sv_lines if ((v==v) and (v <= ml)) ]
                                
                        svdf = pd.DataFrame(sv_lines, index=fsm.filters['vertical_lines_times'], columns=['FSM']).fillna(0)
                        svdf['RUN2'] = svdf['FSM']
                        #if svdf.at['hochlauf','FSM'] > 0.0:
                        #        svdf.at['hochlauf','RUN2'] = sl.loc.timestamp() - start.timestamp() - np.cumsum(svdf['RUN2'])['starter']
                        #        svdf.at['idle','RUN2'] = svdf.at['idle','FSM'] - (svdf.at['hochlauf','RUN2'] - svdf.at['hochlauf','FSM'])
                        if svdf.at['loadramp','FSM'] > 0.0:
                                svdf.at['loadramp','RUN2'] = pl.loc.timestamp() - startversuch['starttime'].timestamp() - np.cumsum(svdf['RUN2'])['synchronize']
                        with warnings.catch_warnings():
                                warnings.simplefilter("ignore")
                                svdf = pd.concat([
                                        svdf, 
                                        pd.DataFrame.from_dict(
                                                {       'maxload':['-',pl.val],
                                                        'ramp':['-',(pl.val / fsm._e['Power_PowerNominal']) * 100 / svdf.at['loadramp','RUN2']],
                                                        'cumstarttime':[np.cumsum(svdf['FSM'])['loadramp'], np.cumsum(svdf['RUN2'])['loadramp']]
                                                }, 
                                                columns=['FSM','RUN2'],
                                                orient='index')]
                                        )

                        ftitle = f"{fsm._e} ----- Start {startversuch['index']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
                        #display(HTML('<h3>'+ f"{fsm._e} ----- Start {startversuch['index']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}" + '</h3>'))
                        
                        fsm.disp_result(startversuch)
                        al_lines = fsm.disp_alarms(startversuch)
                        w_lines = fsm.disp_warnings(startversuch)

                        fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=p_figsize, style='line', line_width=0)
                        #fsm results
                        dmyplant2.add_dbokeh_vlines(fsm.states_lines(startversuch), fig,line_color='red', line_dash='solid', line_alpha=0.4)
                        
                        #alarms & warnings
                        dmyplant2.add_dbokeh_vlines(al_lines,fig,line_color='purple', line_dash='dashed', line_alpha=1, line_width=2)
                        dmyplant2.add_dbokeh_vlines(w_lines,fig,line_color='brown', line_dash='dashed', line_alpha=1, line_width=2)
                        
                        #fsm run 2 results
                        lcol='blue'
                        dmyplant2.add_dbokeh_vlines([sl.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
                        dmyplant2.add_dbokeh_vlines([sr.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
                        dmyplant2.add_dbokeh_vlines([pl.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
                        dmyplant2.add_dbokeh_vlines([pr.loc], fig,line_color=lcol, line_dash='solid', line_alpha=0.4)
                        bokeh.plotting.show(fig)

print(f"Number of Starts in Result: {rdb.shape[0]}")
rdb[fsm.filters['run2filter_content']].round(2)


In [ ]:
#crawl_result(rda, [i for i in range(20,40)], p_figsize=(17,10))
crawl_result(rdb, 17, p_figsize=(17,10))

In [ ]:
rdc = rda[rda['cumstarttime'] > 300.0]
rdc[fsm.filters['run2filter_content']].round(2)

In [ ]:
crawl_result(rdc, 7, p_figsize=(17,10))

In [ ]:
rdd = rda[(rda['loadramp'] < 100.0) & (rda['maxload'] > 4000.0)]
rdd = rdd.sort_values(by = "index",ascending=True).reset_index(drop='index').round(2)
rdd[['starttime']+fsm.filters['run2filter_content']]

In [ ]:
crawl_result(rdd, 41, p_figsize=(17,10))

In [ ]:
rde = rda[rda.starttime > fsm._e['Commissioning Date']]
dset = [
    {'col':['cumstarttime'],'ylim':(-400,900), 'color':'darkblue'},
    {'col':['synchronize'],'ylim':(-50,650)},
    {'col':['startpreparation'],'ylim':(-600,300)},
    {'col':['hochlauf'],'ylim':(-100,200), 'color':'black'},
    {'col':['loadramp'],'ylim':(-150,900), 'color':'green'},
    {'col':['ramprate'],'ylim':(-3,3)},
    {'col':['maxload'],'ylim':(500,5000) }
]
ftitle = f"{fsm._e}"
fig = dmyplant2.dbokeh_chart(rde, dset, x='starttime', figsize=(16,10) ,title=ftitle);
bokeh.plotting.show(fig)

In [ ]:
rda[['startpreparation','starter','hochlauf','idle','synchronize','cumstarttime','loadramp','targetoperation']].hist(figsize = (20,12), bins=80, layout=(3,3));
#rda.hist(figsize = (20,12), bins=100, layout=(4,2));

In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)

In [ ]:
any(fsm._messages['name'] == '9047')

In [ ]:
fsm._target_load_message